## Figure 1 in _Poro-viscoelastic tidal heating of Io_

In [1]:
using Pkg
Pkg.instantiate()

In [2]:
using GLMakie
using Love
using ColorSchemes

IO_OBVS = ((9.33-1.87)*1e13, (9.33+1.87)*1e13)

nr = 100
n = 2
ω = 2 * 2.05e-5
ecc = 0.0048

0.0048

In [ ]:
# Convenience function for returning internal structure based on 
# asthensphere thickness
function get_internal_structure(asthenosphere_thickness)    # Input is in km
    R = 1820.0
    r_cmb = 700.0
    r_crust = 1800.0
    r_asthenosphere = r_crust - asthenosphere_thickness

    return [0, r_cmb, r_asthenosphere, r_crust, R] .* 1e3
end;

# Convenience function to calculate the complex moduli for a 
# poroviscoelastic compacting material
function get_complex_moduli(μ, η, κd, ζ, κs, ω)
    μc = zeros(ComplexF64, length(μ))
    κdc = zero(μc)
    αc = zero(μc)
    
    for i in 1:length(μ) 
        μc[i]  =  isinf(η[i]) ? μ[i] : 1im*ω*μ[i] / (1im*ω + μ[i] / η[i])
        κdc[i] =  isinf(ζ[i]) ? κd[i] : 1im*ω*κd[i] / (1im*ω + κd[i] / ζ[i])
        αc[i]  = 1.0 - κdc[i] / κs[i]
    end
    
    return μc, κdc, αc
end;

get_complex_moduli (generic function with 1 method)

In [4]:
# Solid material parameters -------------------------------------------
ρs = [7649, 3300., 3300, 3000]   # Solid density
# μ = [60, 60, 60, 60] .* 1e9     # Shear modulus
μ = [60, 60, 60, 60] .* 1e9     # Shear modulus
κs = [100e9, 200e9, 200e9, 200e9].*1e30  # Solid bulk modulus
η = [1e25, 1e25, 1e14, 1e25]    # Solid shear viscosity
ζ = [1e25, 1e25, 1e15, 1e25]    # Solid compaction viscosity

# Liquid/two-phase material parameters --------------------------------
ρl = [0, 0, 3300., 0]                # Liquid density
κl = [100e9, 0, 100e9, 0] *1e20   # Liquid bulk modulus
κd = 0.99κs
κd[3] = 200e9
k = [0, 0, 1e-7, 0]                 # Permeability
# α = 1.0 .- κd ./ κs                    # Biot's modulus
α = [0.0, 0.0, 1.0, 0.0]


ηl = [0, 0, 1, 0]                 # Liquid viscosity
ϕ =  [0, 0,0.1, 0]           # Porosity

ρ = (1 .- ϕ) .* ρs + ϕ .* ρl        # Bulk density

D = 300                            # Asthenosphere thickness in km
r = get_internal_structure(D)       # Internal structure

R = r[end]                          # Mean radius 
rr = expand_layers(r, nr=200)        # Discretize primary layers into secondary layers
g = get_g(rr, ρ);                   # Calculate g at each secondary layer#


define_spherical_grid(20.0);

clats = Love.clats 
lons =  Love.lons

nlats = length(clats)
nlons = length(lons);

First we will calculate the global heating rates for the top panel

ηx = 10 .^ collect(10:0.2:19);
Mx = 10 .^ collect(-8:0.2:-3);
ζx = 10 .^ collect(11:0.2:20);

In [ ]:
k[3] = 1e-7
E_Shear = zero(ηx)
for i in eachindex(ηx) 
    η[3] = ηx[i]
    
    μc, κdc, αc = get_complex_moduli(μ, η, κd, ζ, κs, ω)
    y1 = compute_y(rr, ρ, g, μc, κs, ω, ρl, κl, κd, α, ηl, ϕ, k);

    Shear, Bulk, Darcy =  get_heating_profile(conj.(y1), rr, ρ, g, μc, κs, ω, ρl, κl, κd, α, ηl, ϕ, k, ecc; lay=3)
    
    E_Shear[i] = Shear[1][3]
end

η[3] = 1e14
E_Bulk = zero(ζx)
for i in eachindex(ζx) 
    ζ[3] = ζx[i]
    
    μc, κdc, αc = get_complex_moduli(μ, η, κd, ζ, κs, ω)
    y1 = compute_y(rr, ρ, g, μ, κs, ω, ρl, κl, κdc, αc, ηl, ϕ, k);

    Shear, Bulk, Darcy =  get_heating_profile(conj.(y1), rr, ρ, g, μ, κs, ω, ρl, κl, κdc, αc, ηl, ϕ, k, ecc; lay=3)

    E_Bulk[i] = Bulk[1][3]
end

ζ[3] = 1e15
E_Darcy = zero(Mx);
for i in eachindex(Mx) 
    k[3] = Mx[i]
    
    μc, κdc, αc = get_complex_moduli(μ, η, κd, ζ, κs, ω)
    y1 = compute_y(rr, ρ, g, μ, κs, ω, ρl, κl, κdc, αc, ηl, ϕ, k);

    Shear, Bulk, Darcy =  get_heating_profile(conj.(y1), rr, ρ, g, μ, κs, ω, ρl, κl, κdc, αc, ηl, ϕ, k, ecc; lay=3)
    
    E_Darcy[i] = Darcy[1][3]
end

In [7]:
xc = zeros( (size(rr)[1], size(rr)[2], size(clats)[1], size(lons)[2] ) );
yc = zero(xc);
zc = zero(xc);

In [8]:
function sph2cart(r, clat, lon)
    x = r.*sin.(clat) .* cos.(lon)
    y = r.*sin.(clat) .* sin.(lon)
    z = r.*cos.(clat)

    return x, y, z
end

function sphvec2cartvec(vr, vclat, vphi, clat, lon)
    vx = vr.*sin.(clat).*cos.(lon) .+ vclat.*cos.(clat).*cos.(lon) .- vphi.*sin.(lon)
    vy = vr.*sin.(clat).*sin.(lon) .+ vclat.*cos.(clat).*sin.(lon) .+ vphi.*cos.(lon)
    vz = vr.*cos.(clat)           .- vclat.*sin.(clat) 
    return vx, vy, vz
end

sphvec2cartvec (generic function with 1 method)

In [9]:
function extend2360(arr, lon_dim) # expect lats to be the last dim
    dims = ndims(arr)
    
    if lon_dim==3
        a = arr[:,:,1:1]
    elseif lon_dim==2
        a = arr[:,1:1]
    end    
        
    arr = cat(arr, a, dims=lon_dim)

    return arr
end

extend2360 (generic function with 1 method)

In [10]:

function get_slice(vec, scalar, r, theta, phi, limr, limtheta, limphi; in_plane=true)
    nr = size(r)[1]
    nlats = size(theta)[2]
    nlons = size(phi)[3]

    vr = vec[1]
    vtheta = vec[2]
    vphi = vec[3]

    vr = extend2360(vr, 3)
    vtheta = extend2360(vtheta, 3)
    vphi = extend2360(vphi, 3)

    r = extend2360(r, 3)
    theta = extend2360(theta, 3)
    phi = extend2360(phi, 3)
    phi[:,:,end] .= deg2rad(360.0)
    nlons += 1

    if limr[1]==limr[2]     # constant radius 
        rlvl = limr[1]

        theta2 = findlast(x->x<=deg2rad(limtheta[2]), theta[1,:,1])
        theta1 = findfirst(x->x>=deg2rad(limtheta[1]), theta[1,:,1])
        phi2 = findlast(x->x<=deg2rad(limphi[2]), phi[1,1,:])
        phi1 = findfirst(x->x>=deg2rad(limphi[1]), phi[1,1,:])
        
        lonsG_s = phi[rlvl,theta1:theta2,phi1:phi2]
        clatsG_s = theta[rlvl,theta1:theta2,phi1:phi2]
        rG_s    = r[rlvl,theta1:theta2,phi1:phi2]

        xc_s, yc_s, zc_s = sph2cart(rG_s, clatsG_s, lonsG_s);

        if ndims(scalar) == 2
            scalar = extend2360(scalar, 2)
            scalar_s = scalar[theta1:theta2,phi1:phi2]
        else
            scalar = extend2360(scalar, 3)
            scalar_s = scalar[rlvl,theta1:theta2,phi1:phi2]
        end

        
        vr_s = vr[rlvl, theta1:theta2,phi1:phi2]
        vtheta_s = vtheta[rlvl, theta1:theta2,phi1:phi2]
        vphi_s = vphi[rlvl, theta1:theta2,phi1:phi2]

        vx_s, vy_s, vz_s = sphvec2cartvec(vr_s .* !in_plane, vtheta_s, vphi_s, clatsG_s, lonsG_s) 

        pos_xyz = (xc_s, yc_s, zc_s)
        pos_sph = (rG_s, clatsG_s, lonsG_s)
        vec_xyz = (vx_s, vy_s, vz_s);
        
        return vec_xyz, scalar_s, pos_xyz, pos_sph  

    elseif limphi[1]==limphi[2]     # constant longitude 
        philvl = findfirst(x->x>=deg2rad(limphi[1]), phi[1,1,:])

        theta2 = findlast(x->x<=deg2rad(limtheta[2]), theta[1,:,1])
        theta1 = findfirst(x->x>=deg2rad(limtheta[1]), theta[1,:,1])
        r2 = findlast(x->x<=limr[2], r[:,1,1])
        r1 = findfirst(x->x>=limr[1], r[:,1,1])
        
        lonsG_s = phi[:,theta1:theta2,philvl]
        clatsG_s = theta[:,theta1:theta2,philvl]
        rG_s    = r[:,theta1:theta2,philvl]

        xc_s, yc_s, zc_s = sph2cart(rG_s, clatsG_s, lonsG_s);

        scalar = extend2360(scalar, 3)
        scalar_s = scalar[:,theta1:theta2,philvl]

        vr_s = vr[:,theta1:theta2,philvl]
        vtheta_s = vtheta[:,theta1:theta2,philvl]
        vphi_s = vphi[:,theta1:theta2,philvl]

        vx_s, vy_s, vz_s = sphvec2cartvec(vr_s, vtheta_s, vphi_s .* !in_plane, clatsG_s, lonsG_s)

        pos_xyz = (xc_s, yc_s, zc_s)
        pos_sph = (rG_s, clatsG_s, lonsG_s)
        vec_xyz = (vx_s, vy_s, vz_s);

        return vec_xyz, scalar_s, pos_xyz, pos_sph  
    elseif limtheta[1]==limtheta[2]     # constant latitude
        thetalvl = findfirst(x->x>=deg2rad(limtheta[1]), theta[1,:,1])

        phi2 = findlast(x->x<=deg2rad(limphi[2]), phi[1,1,:])
        phi1 = findfirst(x->x>=deg2rad(limphi[1]), phi[1,1,:])
        # r2 = findlast(x->x<=limr[2], r[:,1,1])
        # r1 = findfirst(x->x>=limr[1], r[:,1,1])
        
        lonsG_s = phi[:,thetalvl,phi1:phi2]
        clatsG_s = theta[:,thetalvl,phi1:phi2]
        rG_s    = r[:,thetalvl,phi1:phi2]

        xc_s, yc_s, zc_s = sph2cart(rG_s, clatsG_s, lonsG_s);

        scalar = extend2360(scalar, 3)
        scalar_s = scalar[:,thetalvl,phi1:phi2]

        vr_s = vr[:,thetalvl,phi1:phi2]
        vtheta_s = vtheta[:,thetalvl,phi1:phi2]
        vphi_s = vphi[:,thetalvl,phi1:phi2]

        vx_s, vy_s, vz_s = sphvec2cartvec(vr_s, vtheta_s .* !in_plane, vphi_s , clatsG_s, lonsG_s)

        pos_xyz = (xc_s, yc_s, zc_s)
        pos_sph = (rG_s, clatsG_s, lonsG_s)
        vec_xyz = (vx_s, vy_s, vz_s);

        return vec_xyz, scalar_s, pos_xyz, pos_sph  
    end


end

function create_arrow_data(xyz, v; skipx=1, skipy=1)
    ps = [Point3f(p[1], p[2], p[3]) for p in zip(xyz[1][1:skipx:end,1:skipy:end][:], xyz[2][1:skipx:end,1:skipy:end][:], xyz[3][1:skipx:end,1:skipy:end][:])]
    vs = [Vec3f(p[1], p[2], p[3]) for p in zip(v[1][1:skipx:end,1:skipy:end][:], v[2][1:skipx:end,1:skipy:end][:], v[3][1:skipx:end,1:skipy:end][:])]

    return ps, vs
end


create_arrow_data (generic function with 1 method)

In [ ]:
function turn_off_axes(axes)
    for axis in axes
        axis.yticksvisible=false
        axis.ygridvisible=false
        axis.yspinesvisible=false
        axis.yticklabelsvisible=false
        axis.ylabelvisible=false
        axis.xticksvisible=false
        axis.xgridvisible=false
        axis.xspinesvisible=false
        axis.xticklabelsvisible=false
        axis.xlabelvisible=false
        axis.zticksvisible=false
        axis.zgridvisible=false
        axis.zspinesvisible=false
        axis.zticklabelsvisible=false
        axis.zlabelvisible=false
    end
end

function add_tidal_axis!(ax)
    tide_arr_pos = [Point3f(1.0, 0.0, 0.0)]
    tide_arr_dir = [Vec3f(1.0, 0.0, 0.0)]
    tide_arr_length = 0.2
    tide_arr_headsize = 0.08
    tide_arr_linewidth = 0.05


    arrows!(ax, tide_arr_pos, tide_arr_dir,
        color=(:red,0.8), 
        linewidth=tide_arr_linewidth, 
        lengthscale=tide_arr_length, 
        arrowsize = Vec3f(1.0, 1.0, 1.0) .* tide_arr_headsize)


end

function plot_lines(ax)
    lons = 0.0
    clats = deg2rad.(range(0.0, 180., length=201))
    r = 1.0

    x, y, z = sph2cart(r, clats, lons)
    points = [Point3f(x[i], y[i], z[i]) for i in eachindex(clats)]

    lines!(ax, points, color=:black, linewidth=0.8)

    lons = deg2rad.(280.0)
    clats = deg2rad.(range(0.0, 180., length=201))
    r = 1.0

    x, y, z = sph2cart(r, clats, lons)
    points = [Point3f(x[i], y[i], z[i]) for i in eachindex(clats)]

    lines!(ax, points, color=:black, linewidth=0.8)

    shift = 2e-2
    lons = deg2rad(0.0 - shift)
    clats = deg2rad.(range(0.0, 180., length=201))
    r = rb/R + shift

    x, y, z = sph2cart(r, clats, lons)
    points = [Point3f(x[i], y[i], z[i]) for i in eachindex(clats)]

    lines!(ax, points, color=:black, linewidth=1.5)

    lons = deg2rad.(280.0+shift)
    clats = deg2rad.(range(0.0, 180., length=201))
    r = rb/R+shift

    x, y, z = sph2cart(r, clats, lons)
    points = [Point3f(x[i], y[i], z[i]) for i in eachindex(clats)]

    lines!(ax, points, color=:black, linewidth=1.5)

end

add_tidal_axis! (generic function with 1 method)

In [12]:
# Solid material parameters -------------------------------------------
ρs = [7649, 3300, 3300, 3000]   # Solid density
μ = [60, 60, 60, 60] .* 1e9     # Shear modulus
κs = [200e9, 200e9, 200e9, 200e9].*1e30  # Solid bulk modulus
η = [1e25, 1e25, 1e14, 1e25]    # Solid shear viscosity
ζ = [1e25, 1e25, 1e15, 1e25]    # Solid compaction viscosity

# Liquid/two-phase material parameters --------------------------------
ρl = [0, 0, 3300, 0]                # Liquid density
κl = [0, 0, 100e9, 0]*1e20   # Liquid bulk modulus
κd = [0, 0, 200e9, 0]                         # Drained bulk modulus
k = [0, 0, 1e-5, 0]                 # Permeability
α = 1.0 .- κd ./ κs                    # Biot's modulus

ηl = [0, 0, 1, 0]                 # Liquid viscosity
ϕ =  [0, 0, 0.1, 0]           # Porosity

ρ = (1 .- ϕ) .* ρs + ϕ .* ρl        # Bulk density

D = 300                            # Asthenosphere thickness in km
r = get_internal_structure(D)       # Internal structure


μc, κdc, αc = get_complex_moduli(μ, η, κd, ζ, κs, ω)

R = r[end]                          # Mean radius 
rr = expand_layers(r, nr=nr)        # Discretize primary layers into secondary layers
g = get_g(rr, ρ);                   # Calculate g at each secondary layer#

define_spherical_grid(5.0);

clats = Love.clats 
lons =  Love.lons

nlats = length(clats)
nlons = length(lons);


In [13]:
rr2D = rr[1:end-1,3]
rr2 = rr2D[:]
R = rr2D[end,end]
rb = rr2D[1,1]

clatsG = zeros(length(rr2), length(clats), length(lons))
lonsG  = zero(clatsG)
rG     = zero(clatsG)
for i in 1:length(rr2)
    clatsG[i,:,:] .= clats .* (0 .* lons .+ 1)
    lonsG[i,:,:]  .= (0 .* clats .+ 1) .* lons
    rG[i,:,:]     .= rr2[i]/R
end

xc, yc, zc = sph2cart(rG, clatsG, lonsG);#



In [14]:
U22E =  7/8 * ω^2*R^2*ecc 
U22W = -1/8 * ω^2*R^2*ecc 
U20  = -3/2 * ω^2*R^2*ecc 
ωt = 0.0

κd[3] = 1e20
y1 = compute_y(rr, ρ, g, μc, κs, ω, ρl, κl, κd, α, ηl, ϕ, k);

disps_22, ϵs, σs, ps, d_disps =  get_solution(y1, 2, 2, rr, ρ, g, μc, κs, ω, ρl, κl, κd, α, ηl, ϕ, k);
disps_20, ϵs, σs, ps, d_disps =  get_solution(y1, 2, 0, rr, ρ, g, μc, κs, ω, ρl, κl, κd, α, ηl, ϕ, k);
Eμ, Eμ_vol, Eκ, Eκ_vol, El, El_vol =  get_heating_map(conj.(y1), rr, ρ, g, μc, κs, ω, ρl, κl, κd, α, ηl, ϕ, k, ecc; vol=true)

disps = (U22E*disps_22 + U20*disps_20) .+ U22W*conj.(disps_22);
disps = 0.5real.(disps .* exp(-1im * ωt) .+ conj.(disps) .* exp(1im * ωt));

vr = permutedims(disps[:,:,1,:,3], [3, 1, 2])
vtheta = permutedims(disps[:,:,2,:,3], [3, 1, 2])
vphi = permutedims(disps[:,:,3,:,3], [3, 1, 2])

κd[3] = 200e9
y1 = compute_y(rr, ρ, g, μ, κs, ω, ρl, κl, κdc, αc, ηl, ϕ, k);
y1 = convert( Array{ComplexF64}, y1)

disps_22, ϵs, σs, ps, d_disps =  get_solution(y1, 2, 2, rr, ρ, g, μ, κs, ω, ρl, κl, κdc, αc, ηl, ϕ, k);
disps_20, ϵs, σs, ps, d_disps =  get_solution(y1, 2, 0, rr, ρ, g, μ, κs, ω, ρl, κl, κdc, αc, ηl, ϕ, k);

dum, dum_vol, Eκ, Eκ_vol, El, El_vol =  get_heating_map(conj.(y1), rr, ρ, g, μ, κs, ω, ρl, κl, κdc, αc, ηl, ϕ, k, ecc; vol=true)

disps = (U22E*disps_22 + U20*disps_20) .+ U22W*conj.(disps_22);
disps = 0.5real.(disps .* exp(-1im * ωt) .+ conj.(disps) .* exp(1im * ωt));

vr_c = permutedims(disps[:,:,1,:,3], [3, 1, 2])
vtheta_c = permutedims(disps[:,:,2,:,3], [3, 1, 2])
vphi_c = permutedims(disps[:,:,3,:,3], [3, 1, 2])


κd[3] = 200e9
k[3] = 1e-5
y1 = compute_y(rr, ρ, g, μ, κs, ω, ρl, κl, κd, α, ηl, ϕ, k);

disps, ϵs, σs, ps, d_disps_22 =  get_solution(y1, 2, 2, rr, ρ, g, μ, κs, ω, ρl, κl, κd, α, ηl, ϕ, k);
disps, ϵs, σs, ps, d_disps_20 =  get_solution(y1, 2, 0, rr, ρ, g, μ, κs, ω, ρl, κl, κd, α, ηl, ϕ, k);
dum, dum_vol, dum, dum_vol, El, El_vol =  get_heating_map(conj.(y1), rr, ρ, g, μ, κs, ω, ρl, κl, κd, α, ηl, ϕ, k, ecc; vol=true)

d_disp = (U22E*d_disps_22 + U20*d_disps_20) .+ U22W*conj.(d_disps_22);
d_disp = 0.5real.(d_disp .* exp(-1im * ωt) .+ conj.(d_disp) .* exp(1im * ωt));

qr = permutedims(d_disp[:,:,1,:,3], [3, 1, 2])
qtheta = permutedims(d_disp[:,:,2,:,3], [3, 1, 2])
qphi = permutedims(d_disp[:,:,3,:,3], [3, 1, 2])


El_vol = El_vol[:,:,:,3]
El_vol = permutedims(El_vol, [3, 1, 2])
Eμ_vol = Eμ_vol[:,:,:,3]
Eμ_vol = permutedims(Eμ_vol, [3, 1, 2])
Eκ_vol = Eκ_vol[:,:,:,3]
Eκ_vol = permutedims(Eκ_vol, [3, 1, 2]);



In [ ]:
star_size = 14

# screen = GLMakie.Screen(float=true)
Makie.inline!(false)

scal = 1
f = GLMakie.Figure(size = (1150*scal, 590*scal))

gc = f[1:5, 1] = GridLayout()

g1 = f[1:5, 2] = GridLayout()
g2 = f[1:5, 3] = GridLayout()
g3 = f[1:5, 4] = GridLayout()

g4 = f[5:7, 2] = GridLayout()
g5 = f[5:7, 3] = GridLayout()
g6 = f[5:7, 4] = GridLayout()


elev = deg2rad(15.0)
azim = deg2rad(-35)

ax1 = Axis3(g1[1,1], aspect = :equal, viewmode = :fit, azimuth=azim, elevation=elev, protrusions = -0, title="a) Shear", titlegap=-40, titlesize=20)
ax2 = Axis3(g2[1,1], aspect = :equal, viewmode = :fit, azimuth=azim, elevation=elev, protrusions = -0, title="b) Compaction", titlegap=-40, titlesize=20)
ax3 = Axis3(g3[1,1], aspect = :equal, viewmode = :fit, azimuth=azim, elevation=elev, protrusions = -0, title="c) Darcy flow", titlegap=-40, titlesize=20)

ax4 = Axis(g4[1,1], alignmode=Inside())
ax5 = Axis(g5[1,1])
ax6 = Axis(g6[1,1])



lim = 0.8
lims = ((-lim, lim), (-lim, lim), (-lim, lim))

GLMakie.limits!(ax1, lims[1], lims[2], lims[3])
GLMakie.limits!(ax2, lims[1], lims[2], lims[3])
GLMakie.limits!(ax3, lims[1], lims[2], lims[3])

axes = [ax1, ax2, ax3]

axes2 = [ax4, ax5, ax6]


bandc = "red"
alphac = 0.5
η_fid = 1e14
band!(ax4, ηx, ones(length(ηx)) .* IO_OBVS[1], ones(length(ηx)) .* IO_OBVS[2], color=bandc, alpha=alphac )
η_fid_ind = findfirst(x -> x==η_fid, ηx)
GLMakie.lines!(ax4, ηx, E_Shear, linewidth=2)
GLMakie.scatter!(ax4, η_fid, E_Shear[η_fid_ind], marker=:star5, color="red", markersize=star_size)

ζ_fid = 1e15
band!(ax5, ζx, ones(length(ζx)) .* IO_OBVS[1], ones(length(ζx)) .* IO_OBVS[2], color=bandc, alpha=alphac )
ζ_fid_ind = findfirst(x -> x==ζ_fid, ζx)
GLMakie.lines!(ax5, ζx, E_Bulk, linewidth=2)
GLMakie.scatter!(ax5, ζ_fid, E_Bulk[ζ_fid_ind], marker=:star5, color="red", markersize=star_size)

M_fid = 1e-5
band!(ax6, Mx, ones(length(Mx)) .* IO_OBVS[1], ones(length(Mx)) .* IO_OBVS[2], color=bandc, alpha=alphac )
M_fid_ind = findfirst(x -> x==M_fid, Mx)
GLMakie.lines!(ax6, Mx, E_Darcy, linewidth=2)
GLMakie.scatter!(ax6, M_fid, E_Darcy[M_fid_ind], marker=:star5, color="red", markersize=star_size)

ax4.ylabel = "Dissipated Power [W]"
ax4.xlabel = "Shear viscosity, η [Pa s]"
ax5.xlabel = "Compaction viscosity, ζ [Pa s]"
ax6.xlabel = "Mobility, Mᵩ [m² Pa⁻¹ s⁻¹]"

for axis in axes2
    axis.xscale = log10
    axis.yscale = log10
    axis.yticks = LogTicks(3:2:16)
    axis.yticklabelsize = 17
    axis.xticklabelsize = 17
    axis.xlabelsize = 18
    axis.ylabelsize = 18
end
ax6.xticks = LogTicks(-8:1:-3)
linkyaxes!(ax4, ax5, ax6)

ax5.yticksvisible = false
ax6.yticksvisible = false
ax5.yticklabelsvisible = false
ax6.yticklabelsvisible = false

ax4.alignmode = Mixed(left=-60)

colsize!(f.layout, 1, Auto(0.11))

text!(ax5, (2e11, 1e14),
    text = "Io's heating rate",visible=true, fontsize=14, color=(:red, alphac),)

arr_color = (:black, 0.8);
arr_color = (0.0, 0.0, 0.0, 0.8);
arr_width = 0.0125

alpha_c = 1.0

cmap = ColorSchemes.lipari
cmap = ColorScheme(get(cmap, range(0.15, 1.0, length=256)))

shading=NoShading;

NoShading::ShadingAlgorithm = 0

In [ ]:
rlvl = 100

in_plane=true
v_s1, Eμ_map_s1, xyz_s1, sph_s1 = get_slice((vr, vtheta, vphi), 
                                            Eμ_vol, rG, clatsG, lonsG, 
                                            (rlvl, rlvl), (0, 180.), (5., 280.);in_plane=in_plane)

ps_s1, vs_s1 = create_arrow_data(xyz_s1, v_s1; skipx=2, skipy=2)

v_s1, Eμ_map_s1, xyz_s1, sph_s1 = get_slice((vr, vtheta, vphi), 
                                            Eμ_vol, rG, clatsG, lonsG, 
                                            (rlvl, rlvl), (0, 180.), (0, 280.);in_plane=in_plane)

rlvl = 1
v_s2, Eμ_map_s2, xyz_s2, sph_s2 = get_slice((vr, vtheta, vphi), 
                                            Eμ_vol, rG, clatsG, lonsG, 
                                            (rlvl, rlvl), (0, 180.), (280., 360.);in_plane=in_plane)

ps_s2, vs_s2 = create_arrow_data(xyz_s2, v_s2; skipx=2, skipy=2)

r_bot, r_top = (rr2[1]/R, rr2[end]/R)
v_s3, Eμ_s3, xyz_s3, sph_s3 = get_slice((0.6vr, vtheta, vphi), 
                                            Eμ_vol, rG, clatsG, lonsG, 
                                            (r_bot, r_top), (0, 180.), (0.0, 0.0);in_plane=in_plane)

ps_s3, vs_s3 = create_arrow_data(xyz_s3, v_s3; skipx=40, skipy=2)

v_s4, Eμ_s4, xyz_s4, sph_s4 = get_slice((0.6vr, vtheta, vphi), 
                                            Eμ_vol, rG, clatsG, lonsG, 
                                            (r_bot, r_top), (0, 180.), (280.0, 280.0);in_plane=in_plane)

ps_s4, vs_s4 = create_arrow_data(xyz_s4, v_s4; skipx=40, skipy=2)

ax = ax1
empty!(ax1)

# cmap = (:lipari, 0.85)#olorSchemes.lipari#roma

Eμ_lims = extrema(Eμ_vol)#(0, maximum(Eμ_vol))

GLMakie.surface!(ax, xyz_s1[1], xyz_s1[2], xyz_s1[3], color=Eμ_map_s1, colormap=cmap, colorrange=Eμ_lims, alpha=alpha_c, shading=shading)
s = GLMakie.surface!(ax, xyz_s2[1], xyz_s2[2], xyz_s2[3], color=Eμ_map_s2, colormap=cmap, colorrange=Eμ_lims, alpha=alpha_c, shading=shading)
GLMakie.surface!(ax, xyz_s3[1], xyz_s3[2], xyz_s3[3], color=Eμ_s3, colormap=cmap, colorrange=Eμ_lims, alpha=alpha_c, shading=shading)
GLMakie.surface!(ax, xyz_s4[1], xyz_s4[2], xyz_s4[3], color=Eμ_s4, colormap=cmap, colorrange=Eμ_lims, alpha=alpha_c, shading=shading)

arrows!(ax, 
    vcat(ps_s1, ps_s2, ps_s3, ps_s4), vcat(vs_s1, vs_s2, vs_s3, vs_s4), fxaa=true, # turn on anti-aliasing
    linewidth=arr_width, arrowsize = Vec3f(0.2, 0.2, 0.2) .* 0.15,# color = arr_color,
    align = :center, lengthscale=0.03)


add_tidal_axis!(ax)

turn_off_axes(axes)

Colorbar(gc[1,5:6], 
         label = "Normalised Heating Rate", 
         flipaxis=false, 
         height=130, 
         alignmode=Mixed(left=0),
         limits=(0,1),
         colormap=cmap,
         ticklabelsize=17,
         labelsize=18,
        #  alpha=alpha_c
         )

colgap!(f.layout, 0)

plot_lines(ax)

# display(f)


Lines{Tuple{Vector{Point{3, Float32}}}}

In [ ]:
rlvl = 100

q_s1, El_map_s1, xyz_s1, sph_s1 = get_slice((qr, qtheta, qphi), 
                                            El_vol, rG, clatsG, lonsG, 
                                            (rlvl, rlvl), (0, 180.), (0, 280.0); in_plane=false)

ps_s1, qs_s1 = create_arrow_data(xyz_s1, q_s1; skipx=2, skipy=2)

rlvl = 1
q_s2, El_map_s2, xyz_s2, sph_s2 = get_slice((qr, qtheta, qphi), 
                                            El_vol, rG, clatsG, lonsG, 
                                            (rlvl, rlvl), (0, 180.), (280.0, 360.); in_plane=false)

ps_s2, qs_s2 = create_arrow_data(xyz_s2, q_s2; skipx=2, skipy=2)

r_bot, r_top = (rr2[1]/R, rr2[end]/R)
q_s3, El_s3, xyz_s3, sph_s3 = get_slice((qr, qtheta, qphi), 
                                            El_vol, rG, clatsG, lonsG, 
                                            (r_bot, r_top), (0, 180.), (0.0, 0.0); in_plane=false)

ps_s3, qs_s3 = create_arrow_data(xyz_s3, q_s3; skipx=40, skipy=2)

q_s4, El_s4, xyz_s4, sph_s4 = get_slice((qr, qtheta, qphi), 
                                            El_vol, rG, clatsG, lonsG, 
                                            (r_bot, r_top), (0, 180.), (280.00, 280.00); in_plane=false)

ps_s4, qs_s4 = create_arrow_data(xyz_s4, q_s4; skipx=40, skipy=2)

ax = ax3
empty!(ax3)

El_lims = (0, maximum(El_vol))

GLMakie.surface!(ax, xyz_s1[1], xyz_s1[2], xyz_s1[3], color=El_map_s1, colormap=cmap, shading=shading, colorrange=El_lims, alpha=alpha_c)
GLMakie.surface!(ax, xyz_s2[1], xyz_s2[2], xyz_s2[3], color=El_map_s2, colormap=cmap, shading=shading, colorrange=El_lims, alpha=alpha_c)
GLMakie.surface!(ax, xyz_s3[1], xyz_s3[2], xyz_s3[3], color=El_s3, colormap=cmap, shading=shading, colorrange=El_lims, alpha=alpha_c)
GLMakie.surface!(ax, xyz_s4[1], xyz_s4[2], xyz_s4[3], color=El_s4, colormap=cmap, shading=shading, colorrange=El_lims, alpha=alpha_c)


headscale = 0.15
lengthscale = 0.065

arrows!(ax, 
    vcat(ps_s1, ps_s2, ps_s3, ps_s4), vcat(qs_s1, qs_s2, qs_s3, qs_s4), fxaa=true, # turn on anti-aliasing
    linewidth = arr_width, arrowsize = Vec3f(0.2, 0.2, 0.2) .* headscale,
    align = :automatic, lengthscale=lengthscale#, axis=(type=Axis3,)
)

add_tidal_axis!(ax)

turn_off_axes(axes)

plot_lines(ax)

colgap!(f.layout, 0)

In [ ]:
rlvl = 100

in_plane = false
q_s1, Eκ_map_s1, xyz_s1, sph_s1 = get_slice((vr_c .- vr, vtheta_c .- vtheta, vphi_c .- vphi), #(err, ett, epp), 
                                            Eκ_vol, rG, clatsG, lonsG, 
                                            (rlvl, rlvl), (0, 180.), (0, 280.0); in_plane=in_plane)

ps_s1, qs_s1 = create_arrow_data(xyz_s1, q_s1; skipx=2, skipy=2)

rlvl = 1
q_s2, Eκ_map_s2, xyz_s2, sph_s2 = get_slice((vr_c .- vr, vtheta_c .- vtheta, vphi_c .- vphi), #(err, ett, epp),  
                                            Eκ_vol, rG, clatsG, lonsG, 
                                            (rlvl, rlvl), (0, 180.), (280.0, 360.); in_plane=in_plane)

ps_s2, qs_s2 = create_arrow_data(xyz_s2, q_s2; skipx=2, skipy=2)

r_bot, r_top = (rr2[1]/R, rr2[end]/R)
q_s3, Eκ_s3, xyz_s3, sph_s3 = get_slice((vr_c .- vr, vtheta_c .- vtheta, vphi_c .- vphi), #(err, ett, epp),  
                                            Eκ_vol, rG, clatsG, lonsG, 
                                            (r_bot, r_top), (0, 180.), (0.0, 0.0); in_plane=in_plane)

ps_s3, qs_s3 = create_arrow_data(xyz_s3, q_s3; skipx=40, skipy=2)

q_s4, Eκ_s4, xyz_s4, sph_s4 = get_slice((vr_c .- vr, vtheta_c .- vtheta, vphi_c .- vphi), #(err, ett, epp),  
                                            Eκ_vol, rG, clatsG, lonsG, 
                                            (r_bot, r_top), (0, 180.), (280.00, 280.00); in_plane=in_plane)

ps_s4, qs_s4 = create_arrow_data(xyz_s4, q_s4; skipx=40, skipy=2)

ax = ax2
empty!(ax)

Eκ_lims = (0, maximum(Eκ_vol))

GLMakie.surface!(ax, xyz_s1[1], xyz_s1[2], xyz_s1[3], color=Eκ_map_s1, colormap=cmap, shading=shading, colorrange=Eκ_lims, alpha=alpha_c)
GLMakie.surface!(ax, xyz_s2[1], xyz_s2[2], xyz_s2[3], color=Eκ_map_s2, colormap=cmap, shading=shading, colorrange=Eκ_lims, alpha=alpha_c)
GLMakie.surface!(ax, xyz_s3[1], xyz_s3[2], xyz_s3[3], color=Eκ_s3, colormap=cmap, shading=shading, colorrange=Eκ_lims, alpha=alpha_c)
GLMakie.surface!(ax, xyz_s4[1], xyz_s4[2], xyz_s4[3], color=Eκ_s4, colormap=cmap, shading=shading, colorrange=Eκ_lims, alpha=alpha_c)


headscale = 0.05
lengthscale = 2.0

add_tidal_axis!(ax)
turn_off_axes(axes)
plot_lines(ax)

colgap!(f.layout, 20)
rowgap!(f.layout, 40)


trim!(gc)
save("./figures/fig1_io_overview.png", f; px_per_unit=10)

# display(f)